In [1]:
import numpy as np
import time
import copy
import random

In [2]:
import gym
import numpy.ma as ma

import torch
import torch.nn as nn

import cv2
import gym.spaces
import collections

import argparse
import time
import torch.optim as optim

from tensorboardX import SummaryWriter

In [3]:
nSamples = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
machineNames = ['m0','m1','m2','m3','m4']

In [4]:
# p - persistance
# r - ready time

# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r 
# r  >= c - p

# Game - Generate

In [211]:
class ptszGenerator(gym.Env):
    def __init__(self, k, N):
        self.action_space = k
        self.observation_space = N
        
        self.machines = None
        self.tasks = None
        self.observations = None
        
        self.schedule = []
        for i in range(self.action_space):
            self.schedule.append([])
        
        self.taskCounter = 0
    
    def seed(self, seed=None):
        print("seed not implemented")
        pass        
    
    
    def step(self, actions):
        if (actions == self.observation_space).all(): #if action is do nothing then increase current time to allow new machine or new task
            #self.current_time
            #if there is no active task 
            i = self.action_space - 1
            theClosestTaskTime = np.inf
            thereIsActiveTask = False
            current_time = self.observations[-1]
            for j in range(self.observation_space):
                r = self.observations[2*(i+1)+j*3+1]
                done = self.observations[2*(i+1)+j*3+2]
                if not done and r < current_time:
                    #print("There is active task")
                    thereIsActiveTask = True
                    break # there is active task
                elif not done:
                    if r < theClosestTaskTime:
                        theClosestTaskTime = r
            if not thereIsActiveTask:
                self.observations[-1] += theClosestTaskTime
                return self.observations, 0, False, ""
            #else there is active task but there is no active machine
        
            readyTime = []
            for i in range(self.action_space):
                readyTime.append(self.observations[2*i+1])
            #print(f"readyTime: {readyTime} theClosestTaskTime: {theClosestTaskTime} currentTime: {current_time}")
            readyTime = np.array(readyTime)
            try:
                minNextMachine = np.min(readyTime[readyTime > current_time])
            except:
                print("except")
                return self.observations, 0, False, ""
            #print(f"minNextMachine: {minNextMachine}")
          #  if len(readyTime[readyTime > current_time]) != self.action_space:
          #      print(f"len: {len(readyTime[readyTime > current_time])}")
          #      print("ocho we wait to next machine here")
          #      print(f"readyTime: {readyTime} theClosestTaskTime: {theClosestTaskTime} currentTime: {current_time}")
          #      print(f"minNextMachine: {minNextMachine}")
            self.observations[-1] = minNextMachine
            return self.observations, 0, False, ""
        
        #preprocess_action = self.preprocess_action(actions)
        #print(preprocess_action)
        #for i,a in enumerate(preprocess_action):
        for i,a in enumerate(actions):
            if a == self.observation_space: # machine 'a' does nothing
                continue
            self.observations[self.action_space*2 + 3*a + 2] = 1 # mark tasks as done

            current_time = self.observations[-1]
            p,r,_ = self.observations[self.action_space*2 + 3*a: self.action_space*2 + 3*(a+1)] # take r,p,done of selected task
            
            speed = self.observations[2*i]
            self.observations[2*i+1] = current_time + p / speed #ready_time of the current mashine
            
            #print(f"m_{i} took task {a}")
            self.schedule[i].append(a)
            self.taskCounter += 1
            
        done = False
        reward = 0
        if self.taskCounter == self.observation_space:
            done = True
            reward = self.calculateCost()
            
        
        return self.observations, reward, done, ""
    
    def reset(self):
        # returns: observation: machines, tasks, current_time
        #                       [speed, ready_time]*k, [r, p, done]*N, current_time
        # shape: (1,2*k+3*N+1)
        self.taskCounter = 0
        self.machines = None
        self.tasks = None
        self.observations = None
        
        self.schedule = []
        for i in range(self.action_space):
            self.schedule.append([])
        
        timeCounter = 0
        tasks = []
        for _ in range(self.observation_space):
            p = np.random.randint(1, 100)
            r = np.random.randint(timeCounter - 5 if timeCounter - 5 > 0 else 0 ,timeCounter + 2)
            timeCounter += int(p/3)
            
            #ready, persistance, doneBoolean
            tasks.append([p,r,0])
        #random.shuffle(tasks) ///////////////////////////////////////////// to uncomment
        self.tasks = np.array(tasks)
        
        m0 = np.random.randint(30,100)
        m1 = np.random.randint(30,100)
        m2 = np.random.randint(30,35)
        m3 = np.random.randint(30,100)
        m4 = np.random.randint(99,100)
        machines = np.array([ [m0,0], [m1,0], [m2,0] ,[m3,0] ,[m4,0]], dtype="float")
        machines[:,0] =  machines[:,0] /max(machines[:,0])

        index = range(5)
        indicies = np.random.choice( index, 5, replace=False )
        self.machines = machines[indicies]

        mechineTasks = np.concatenate((machines,tasks), axis=None)
        self.observations = np.concatenate((mechineTasks, 0), axis=None)
        
        return self.observations
    
        
    def get_valid_tasks(self):
        readyToGo = np.ones(self.observation_space)
        timeCounter = self.observations[-1]
        for i,obs in enumerate(range(self.action_space*2, self.action_space*2+self.observation_space*3, 3)):
            p,r,done = self.observations[[obs,obs+1,obs+2]]
            if r <= timeCounter and done==0: # task is ready and not done
                readyToGo[i] = 0
                                 
        return readyToGo
                               
    
    def get_valid_machines(self):
        readyMachines = []
        timeCounter = self.observations[-1]
        for i in range(self.action_space):
            speed = self.observations[2*i]
            readyTime = self.observations[2*i+1]
            
            if readyTime <= timeCounter:
                readyMachines.append(i)
        return np.array(readyMachines)
    
    
    def _get_obs(self):
        return self.observations
    
    
    def preprocess_action(self, action):
        # input: actions in shape (n,k)
        #output: actions in shape (n), where each number means id of task to take od sleep for number== -1
        filteredAction = []
        validT = self.get_valid_tasks()
        mask = np.array((*validT, 0)) # 1 means invalid
        
        valid_machines = self.get_valid_machines()        
        for i, j in enumerate(range(0, self.action_space* (self.observation_space+1) , self.observation_space+1 )):            
            a = action[j: j+ (self.observation_space+1)]
            if i not in valid_machines:
                filteredAction.append(self.observation_space)
                continue
            mask_arr = ma.masked_array(a, mask=mask) 
            argmin = mask_arr.argmin()
            
            if argmin == self.observation_space:
                filteredAction.append(argmin)
                continue
            mask[argmin] = 1
            filteredAction.append(argmin)
        return np.array(filteredAction)
        
        
    def print_observations(self):
        for i in range(self.action_space):
            print(f"machine_{i} speed: {round(self.observations[2*i],2)} readyTime: {round(self.observations[2*i+1],2)}")
        for j in range(self.observation_space):
            print(f"task_{j+1} p: {self.observations[2*(i+1)+j*3]} r: {self.observations[2*(i+1)+j*3+1]} done: {self.observations[2*(i+1)+j*3+2]}")
        
        print(self.observations[-1])
    
    
    def sample(self):
        return self.preprocess_action(np.random.rand(self.action_space*(self.observation_space+1))) #+1 because of sleep behaviour
    
    def calculateCost(self):
        cTime = [0]*self.observation_space
        p,r=0,1
        speed,readyTime = 0,1
        calculatedWeight = 0
        timeCounter = 0
        
        for machine in self.machines:
            machine[readyTime] = 0
        
        for key, tasks in enumerate(self.schedule):
            #print(f"m: {key}")
            timeCounter = 0
            #print(f"machine: {key}, {self.machines[key]}")
            sortedTasks = sorted(tasks, key=lambda i: self.tasks[i][1]) # python is really good
            for task in sortedTasks:
                #print(f"task: {task}, {self.tasks[task]}")
                if self.machines[key][readyTime] > timeCounter or self.tasks[task][r] > timeCounter:
                    timeCounter = max(self.machines[key][readyTime],  self.tasks[task][r])
                #print(f"machines[{key}][readyTime]: {self.machines[key][readyTime]}")

                self.machines[key][readyTime] = timeCounter + (self.tasks[task][p] / float(self.machines[key][speed]) )

                #print(f"machines[{key}]['readyTime']: {self.machines[key][readyTime]}")

                cTime[task] = timeCounter + (self.tasks[task][p] / float(self.machines[key][speed]) ) - self.tasks[task][r]
                calculatedWeight += cTime[task]

                #print(f"cTime[task]: {cTime[task]}")
                #print(f"real p: {(self.tasks[task][p] / float(self.machines[key][speed]) )}")
                #print()
         
        #for key, value in enumerate(self.schedule):
        #    print(f"{key} -> {value}")
        #
        #print(calculatedWeight)
        return calculatedWeight / self.observation_space


In [212]:
for i,a in enumerate(range(0,255,51)):
    print(i,a)

0 0
1 51
2 102
3 153
4 204


In [213]:
x = ptszGenerator(5,50)
x.reset()


array([6.86868687e-01, 0.00000000e+00, 9.49494949e-01, 0.00000000e+00,
       3.13131313e-01, 0.00000000e+00, 7.77777778e-01, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 5.90000000e+01, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+01, 1.60000000e+01, 0.00000000e+00,
       3.00000000e+00, 2.30000000e+01, 0.00000000e+00, 1.30000000e+01,
       2.30000000e+01, 0.00000000e+00, 5.20000000e+01, 2.40000000e+01,
       0.00000000e+00, 3.30000000e+01, 4.20000000e+01, 0.00000000e+00,
       1.10000000e+01, 5.40000000e+01, 0.00000000e+00, 7.50000000e+01,
       5.30000000e+01, 0.00000000e+00, 7.50000000e+01, 8.10000000e+01,
       0.00000000e+00, 6.00000000e+00, 1.05000000e+02, 0.00000000e+00,
       8.30000000e+01, 1.06000000e+02, 0.00000000e+00, 4.70000000e+01,
       1.35000000e+02, 0.00000000e+00, 1.30000000e+01, 1.51000000e+02,
       0.00000000e+00, 3.20000000e+01, 1.53000000e+02, 0.00000000e+00,
       8.80000000e+01, 1.62000000e+02, 0.00000000e+00, 8.50000000e+01,
      

In [220]:
x.print_observations()

machine_0 speed: 0.64 readyTime: 0.0
machine_1 speed: 0.89 readyTime: 0.0
machine_2 speed: 0.34 readyTime: 0.0
machine_3 speed: 0.41 readyTime: 0.0
machine_4 speed: 1.0 readyTime: 0.0
task_1 p: 9.0 r: 1.0 done: 0.0
task_2 p: 92.0 r: 0.0 done: 0.0
task_3 p: 59.0 r: 33.0 done: 0.0
task_4 p: 40.0 r: 48.0 done: 0.0
task_5 p: 3.0 r: 66.0 done: 0.0
task_6 p: 2.0 r: 62.0 done: 0.0
task_7 p: 80.0 r: 61.0 done: 0.0
task_8 p: 60.0 r: 90.0 done: 0.0
task_9 p: 14.0 r: 110.0 done: 0.0
task_10 p: 70.0 r: 113.0 done: 0.0
task_11 p: 38.0 r: 134.0 done: 0.0
task_12 p: 32.0 r: 146.0 done: 0.0
task_13 p: 63.0 r: 158.0 done: 0.0
task_14 p: 28.0 r: 177.0 done: 0.0
task_15 p: 59.0 r: 190.0 done: 0.0
task_16 p: 83.0 r: 206.0 done: 0.0
task_17 p: 67.0 r: 232.0 done: 0.0
task_18 p: 37.0 r: 259.0 done: 0.0
task_19 p: 7.0 r: 271.0 done: 0.0
task_20 p: 84.0 r: 273.0 done: 0.0
task_21 p: 9.0 r: 301.0 done: 0.0
task_22 p: 41.0 r: 304.0 done: 0.0
task_23 p: 15.0 r: 313.0 done: 0.0
task_24 p: 91.0 r: 319.0 done: 0.0


In [215]:
o, r , done, _ = x.step(x.sample())
while not done:
    o, r , done, _ = x.step(x.sample())
print(r)

198.2916484891829


In [216]:
x.reset()

array([6.36363636e-01, 0.00000000e+00, 8.88888889e-01, 0.00000000e+00,
       3.43434343e-01, 0.00000000e+00, 4.14141414e-01, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 9.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 9.20000000e+01, 0.00000000e+00, 0.00000000e+00,
       5.90000000e+01, 3.30000000e+01, 0.00000000e+00, 4.00000000e+01,
       4.80000000e+01, 0.00000000e+00, 3.00000000e+00, 6.60000000e+01,
       0.00000000e+00, 2.00000000e+00, 6.20000000e+01, 0.00000000e+00,
       8.00000000e+01, 6.10000000e+01, 0.00000000e+00, 6.00000000e+01,
       9.00000000e+01, 0.00000000e+00, 1.40000000e+01, 1.10000000e+02,
       0.00000000e+00, 7.00000000e+01, 1.13000000e+02, 0.00000000e+00,
       3.80000000e+01, 1.34000000e+02, 0.00000000e+00, 3.20000000e+01,
       1.46000000e+02, 0.00000000e+00, 6.30000000e+01, 1.58000000e+02,
       0.00000000e+00, 2.80000000e+01, 1.77000000e+02, 0.00000000e+00,
       5.90000000e+01, 1.90000000e+02, 0.00000000e+00, 8.30000000e+01,
      

In [217]:
x.sample()

array([50, 50, 50,  1, 50])

In [218]:
x.observations[-1] = 1

In [219]:
x.sample()

array([ 1,  0, 50, 50, 50])

# QGN neural network model

In [223]:
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        # self.conv = nn.Sequential(
        #     nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=4, stride=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1),
        #     nn.ReLU()
        # )

        self.fc = nn.Sequential(
            nn.Linear(input_shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, n_actions)
        )

    def forward(self, x):
        #conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(x.float())

    # def __init__(self, input_shape, n_actions):
    #     super(DQN, self).__init__()

    #     self.conv = nn.Sequential(
    #         nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
    #         nn.ReLU(),
    #         nn.Conv2d(32, 64, kernel_size=4, stride=2),
    #         nn.ReLU(),
    #         nn.Conv2d(64, 64, kernel_size=3, stride=1),
    #         nn.ReLU()
    #     )

    #     conv_out_size = self._get_conv_out(input_shape)
    #     self.fc = nn.Sequential(
    #         nn.Linear(conv_out_size, 512),
    #         nn.ReLU(),
    #         nn.Linear(512, n_actions)
    #     )

    # def _get_conv_out(self, shape):
    #     o = self.conv(torch.zeros(1, *shape))
    #     return int(np.prod(o.size()))

    # def forward(self, x):
    #     conv_out = self.conv(x).view(x.size()[0], -1)
    #     return self.fc(conv_out)

# DQN

In [205]:
DEVICE = False #  True if'cuda'
DEFAULT_ENV_NAME = "TaskScheduler"

MEAN_REWARD_BOUND = 150

GAMMA = 0.99
BATCH_SIZE = 128
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 1000

EPSILON_DECAY_LAST_FRAME = 100000
EPSILON_START = 1.0
EPSILON_FINAL = 0.01


Experience = collections.namedtuple(
    'Experience', field_names=['state', 'action', 'reward',
                               'done', 'new_state'])

In [206]:
class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size,
                                   replace=False)
        states, actions, rewards, dones, next_states = \
            zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), \
               np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), \
               np.array(next_states)

In [207]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            action = env.preprocess_action(q_vals_v[0])
            
            #_, act_v = torch.max(q_vals_v, dim=1)
            #action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward,
                         is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward


In [208]:
def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(np.array(
        states, copy=False)).to(device)
    next_states_v = torch.tensor(np.array(
        next_states, copy=False)).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)


    state_action_values = net(states_v).gather(
        1, actions_v).sum(1)
    
    with torch.no_grad():
        tgt_state_values = tgt_net(next_states_v)#.max(1)[0]
        for i, j in enumerate(range(0, MACHINES* (TASKS+1) , TASKS+1 )):    
            if i==0:
                next_state_values = tgt_state_values[:, j: j+(TASKS+1)].max(1)[0]
            else:
                next_state_values += tgt_state_values[:, j: j+(TASKS+1)].max(1)[0]
        #print(f"next_state_values= {next_state_values}")
        next_state_values[done_mask] = 0.0
        next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + \
                                   rewards_v
    return nn.MSELoss()(state_action_values,
                        expected_state_action_values)

In [ ]:
MACHINES = 5
TASKS = 50
action_space = MACHINES* (TASKS+1)
observation_space = (MACHINES * 2 + TASKS * 3 + 1,)

device = torch.device("cuda" if DEVICE else "cpu")

env = ptszGenerator(MACHINES,TASKS)

net = DQN(observation_space,
                    action_space).to(device) # TODO: take it from the env
tgt_net = DQN(observation_space,
                        action_space).to(device)
writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)
print(net)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START -
                  frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, epsilon, device=device)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        print("%d: done %d games, reward %.3f, "
              "eps %.2f, speed %.2f f/s" % (
            frame_idx, len(total_rewards), m_reward, epsilon,
            speed
        ))
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or best_m_reward > m_reward:
            torch.save(net.state_dict(), DEFAULT_ENV_NAME +
                       "-best_%.0f.dat" % m_reward)
            if best_m_reward is not None:
                print("Best reward updated %.3f -> %.3f" % (
                    best_m_reward, m_reward))
            best_m_reward = m_reward
        if m_reward < MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    states, actions, rewards, dones, next_states = batch
    loss_t = calc_loss(batch, net, tgt_net, device=device)
    loss_t.backward()
    optimizer.step()
writer.close()

DQN(
  (fc): Sequential(
    (0): Linear(in_features=161, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=255, bias=True)
  )
)
88: done 1 games, reward 184.073, eps 1.00, speed 1234.34 f/s
168: done 2 games, reward 247.674, eps 1.00, speed 1364.71 f/s
265: done 3 games, reward 352.808, eps 1.00, speed 1622.37 f/s
343: done 4 games, reward 285.954, eps 1.00, speed 2324.66 f/s
425: done 5 games, reward 272.259, eps 1.00, speed 2293.16 f/s
514: done 6 games, reward 268.287, eps 0.99, speed 2529.45 f/s
606: done 7 games, reward 302.661, eps 0.99, speed 2869.03 f/s
694: done 8 games, reward 304.955, eps 0.99, speed 2921.40 f/s
787: done 9 games, reward 323.106, eps 0.99, speed 3045.50 f/s
869: done 10 games, reward 327.571, eps 0.99, speed 2349.57 f/s
952: done 11 games, reward 327.173, eps 0.99, speed 2529.44 f/s
1034: done 12 games, reward 332.066, eps 0.99, speed 239.

10803: done 127 games, reward 281.891, eps 0.89, speed 87.75 f/s
10889: done 128 games, reward 280.854, eps 0.89, speed 82.23 f/s
10981: done 129 games, reward 279.646, eps 0.89, speed 79.17 f/s
11059: done 130 games, reward 276.429, eps 0.89, speed 86.04 f/s
11152: done 131 games, reward 276.856, eps 0.89, speed 85.34 f/s
11245: done 132 games, reward 279.467, eps 0.89, speed 87.19 f/s
11334: done 133 games, reward 278.916, eps 0.89, speed 84.94 f/s
11417: done 134 games, reward 276.427, eps 0.89, speed 82.99 f/s
11509: done 135 games, reward 275.406, eps 0.88, speed 84.23 f/s
11595: done 136 games, reward 276.669, eps 0.88, speed 83.17 f/s
11681: done 137 games, reward 278.594, eps 0.88, speed 88.69 f/s
11767: done 138 games, reward 276.597, eps 0.88, speed 84.09 f/s
11848: done 139 games, reward 273.421, eps 0.88, speed 81.73 f/s
11931: done 140 games, reward 272.567, eps 0.88, speed 87.27 f/s
12020: done 141 games, reward 270.348, eps 0.88, speed 82.92 f/s
12096: done 142 games, re

21621: done 253 games, reward 268.100, eps 0.78, speed 82.73 f/s
21705: done 254 games, reward 268.900, eps 0.78, speed 83.71 f/s
21782: done 255 games, reward 269.405, eps 0.78, speed 72.23 f/s
21862: done 256 games, reward 269.153, eps 0.78, speed 82.07 f/s
21953: done 257 games, reward 272.799, eps 0.78, speed 84.44 f/s
22032: done 258 games, reward 272.215, eps 0.78, speed 84.86 f/s
22124: done 259 games, reward 273.406, eps 0.78, speed 82.49 f/s
22216: done 260 games, reward 271.356, eps 0.78, speed 82.63 f/s
22302: done 261 games, reward 272.677, eps 0.78, speed 85.64 f/s
22392: done 262 games, reward 273.222, eps 0.78, speed 86.40 f/s
22471: done 263 games, reward 273.334, eps 0.78, speed 84.70 f/s
22552: done 264 games, reward 275.746, eps 0.77, speed 85.66 f/s
22634: done 265 games, reward 276.455, eps 0.77, speed 81.35 f/s
22718: done 266 games, reward 278.648, eps 0.77, speed 84.29 f/s
22804: done 267 games, reward 274.936, eps 0.77, speed 77.51 f/s
22884: done 268 games, re

32443: done 380 games, reward 249.374, eps 0.68, speed 90.71 f/s
32526: done 381 games, reward 248.947, eps 0.67, speed 81.71 f/s
32607: done 382 games, reward 249.895, eps 0.67, speed 94.43 f/s
32694: done 383 games, reward 250.637, eps 0.67, speed 96.98 f/s
32782: done 384 games, reward 250.299, eps 0.67, speed 80.30 f/s
32866: done 385 games, reward 249.914, eps 0.67, speed 91.34 f/s
32956: done 386 games, reward 252.309, eps 0.67, speed 101.25 f/s
33032: done 387 games, reward 248.435, eps 0.67, speed 96.56 f/s
33107: done 388 games, reward 247.838, eps 0.67, speed 94.89 f/s
33195: done 389 games, reward 247.749, eps 0.67, speed 88.79 f/s
33283: done 390 games, reward 249.051, eps 0.67, speed 85.30 f/s
33374: done 391 games, reward 248.834, eps 0.67, speed 97.01 f/s
33461: done 392 games, reward 249.194, eps 0.67, speed 83.66 f/s
33549: done 393 games, reward 247.364, eps 0.66, speed 88.27 f/s
33645: done 394 games, reward 250.875, eps 0.66, speed 98.38 f/s
33724: done 395 games, r

43446: done 507 games, reward 242.796, eps 0.57, speed 99.37 f/s
43527: done 508 games, reward 241.058, eps 0.56, speed 85.04 f/s
43616: done 509 games, reward 240.927, eps 0.56, speed 93.43 f/s
43696: done 510 games, reward 240.473, eps 0.56, speed 95.46 f/s
43789: done 511 games, reward 241.689, eps 0.56, speed 73.16 f/s
43885: done 512 games, reward 242.567, eps 0.56, speed 90.91 f/s
43974: done 513 games, reward 240.307, eps 0.56, speed 72.09 f/s
44050: done 514 games, reward 237.859, eps 0.56, speed 73.03 f/s
44127: done 515 games, reward 237.116, eps 0.56, speed 90.96 f/s
44210: done 516 games, reward 236.151, eps 0.56, speed 89.81 f/s
44298: done 517 games, reward 236.236, eps 0.56, speed 83.85 f/s
44378: done 518 games, reward 237.115, eps 0.56, speed 88.20 f/s
44462: done 519 games, reward 238.696, eps 0.56, speed 94.23 f/s
44551: done 520 games, reward 238.971, eps 0.55, speed 88.84 f/s
44641: done 521 games, reward 239.097, eps 0.55, speed 87.30 f/s
44727: done 522 games, re

...

except
except
except
321037: done 2862 games, reward 296.855, eps 0.01, speed 30.63 f/s
321379: done 2863 games, reward 295.945, eps 0.01, speed 33.31 f/s
321472: done 2864 games, reward 297.312, eps 0.01, speed 35.89 f/s
except
321563: done 2865 games, reward 297.824, eps 0.01, speed 37.11 f/s
except
except
except
except
except
except
except
except
except
except
321657: done 2866 games, reward 296.548, eps 0.01, speed 36.59 f/s
321755: done 2867 games, reward 298.585, eps 0.01, speed 38.82 f/s
321935: done 2868 games, reward 299.401, eps 0.01, speed 38.36 f/s
322090: done 2869 games, reward 297.706, eps 0.01, speed 37.40 f/s
322230: done 2870 games, reward 298.801, eps 0.01, speed 38.79 f/s
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
322344: done 2871 games, reward 295.409, eps 0.01, speed 35.18 f/s
except
except
except
except
except
except
except
except
except
except
except
e

except
except
except
except
except
except
except
328787: done 2918 games, reward 281.769, eps 0.01, speed 38.82 f/s
328896: done 2919 games, reward 282.086, eps 0.01, speed 38.20 f/s
except
except
except
except
except
except
except
except
except
except
except
except
329002: done 2920 games, reward 284.847, eps 0.01, speed 39.87 f/s
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
329188: done 2921 games, rewa

except
336146: done 2975 games, reward 287.780, eps 0.01, speed 36.43 f/s
336239: done 2976 games, reward 287.217, eps 0.01, speed 36.77 f/s
336333: done 2977 games, reward 288.829, eps 0.01, speed 36.81 f/s
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
e

343557: done 3033 games, reward 280.241, eps 0.01, speed 38.44 f/s
343650: done 3034 games, reward 281.661, eps 0.01, speed 35.56 f/s
343743: done 3035 games, reward 282.024, eps 0.01, speed 38.19 f/s
343840: done 3036 games, reward 283.980, eps 0.01, speed 37.63 f/s
343930: done 3037 games, reward 286.355, eps 0.01, speed 38.75 f/s
344019: done 3038 games, reward 288.305, eps 0.01, speed 38.02 f/s
344112: done 3039 games, reward 288.654, eps 0.01, speed 37.39 f/s
344204: done 3040 games, reward 287.139, eps 0.01, speed 39.05 f/s
344294: done 3041 games, reward 287.314, eps 0.01, speed 37.69 f/s
344388: done 3042 games, reward 286.282, eps 0.01, speed 37.98 f/s
344484: done 3043 games, reward 284.386, eps 0.01, speed 37.43 f/s
344576: done 3044 games, reward 283.586, eps 0.01, speed 36.99 f/s
344669: done 3045 games, reward 284.560, eps 0.01, speed 38.11 f/s
344761: done 3046 games, reward 282.481, eps 0.01, speed 36.67 f/s
344857: done 3047 games, reward 282.248, eps 0.01, speed 38.14

351236: done 3111 games, reward 283.917, eps 0.01, speed 38.18 f/s
351327: done 3112 games, reward 283.022, eps 0.01, speed 38.34 f/s
351421: done 3113 games, reward 283.109, eps 0.01, speed 39.17 f/s
351516: done 3114 games, reward 283.013, eps 0.01, speed 38.25 f/s
351606: done 3115 games, reward 281.890, eps 0.01, speed 38.20 f/s
351694: done 3116 games, reward 284.823, eps 0.01, speed 38.72 f/s
351789: done 3117 games, reward 282.913, eps 0.01, speed 38.13 f/s
351881: done 3118 games, reward 284.272, eps 0.01, speed 39.48 f/s
351977: done 3119 games, reward 285.652, eps 0.01, speed 39.17 f/s
352071: done 3120 games, reward 285.820, eps 0.01, speed 37.95 f/s
352165: done 3121 games, reward 285.273, eps 0.01, speed 39.43 f/s
352260: done 3122 games, reward 286.374, eps 0.01, speed 39.32 f/s
352421: done 3123 games, reward 284.783, eps 0.01, speed 38.88 f/s
352520: done 3124 games, reward 285.313, eps 0.01, speed 37.91 f/s
352617: done 3125 games, reward 288.333, eps 0.01, speed 39.31

In [155]:
def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(np.array(
        states, copy=False)).to(device)
    next_states_v = torch.tensor(np.array(
        next_states, copy=False)).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)


    state_action_values = net(states_v).gather(
        1, actions_v).sum(1)
    
    with torch.no_grad():
        tgt_state_values = tgt_net(next_states_v)#.max(1)[0]
        for i, j in enumerate(range(0, MACHINES* (TASKS+1) , TASKS+1 )):    
            if i==0:
                next_state_values = tgt_state_values[:, j: j+(TASKS+1)].max(1)[0]
            else:
                next_state_values += tgt_state_values[:, j: j+(TASKS+1)].max(1)[0]
        #print(f"next_state_values= {next_state_values}")
        next_state_values[done_mask] = 0.0
        next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + \
                                   rewards_v
    return nn.MSELoss()(state_action_values,
                        expected_state_action_values)

In [105]:
for x in nSamples:
    timeCounter = 0
    fileData = []
    fileData.append(x)
    for _ in range(x):
        p = np.random.randint(1, 100)
        r = np.random.randint(timeCounter - 5 if timeCounter - 5 > 0 else 0 ,timeCounter + 2)
        timeCounter += int(p/3)
        
        fileData.append([p,r])
        
    with open(f'PTSZ-instancje/136723/in136723_{x}.txt', "w") as f:
        f.write(f"{str(fileData[0])}\n")
        
        m0 = np.random.randint(50,100)
        m1 = np.random.randint(50,100)
        m2 = np.random.randint(50,60)
        m3 = np.random.randint(50,100)
        m4 = np.random.randint(99,100)
        machines = np.array([m0,m1,m2,m3,m4])
        machines =  machines /max(machines)
        machines = np.random.choice( machines, 5, replace=False )
        f.write(f"{' '.join([str(x) for x in machines])}\n")
        
        instances = fileData[1:]
        random.shuffle(instances)
        for line in instances:            
            f.write(f"{' '.join([str(x) for x in line])}\n")

# Verify

In [5]:
x = 50

In [101]:
data

{1: {'p': 89, 'r': 588, 'c': 1123.578947368421},
 2: {'p': 78, 'r': 131, 'c': 519.4736842105262},
 3: {'p': 36, 'r': 405, 'c': 1369.5263157894738},
 4: {'p': 52, 'r': 363, 'c': 399.0},
 5: {'p': 64, 'r': 441, 'c': 419.0},
 6: {'p': 34, 'r': 8, 'c': 788.0},
 7: {'p': 60, 'r': 685, 'c': 87.35294117647061},
 8: {'p': 79, 'r': 299, 'c': 175.01470588235293},
 9: {'p': 38, 'r': 263, 'c': 772.0},
 10: {'p': 23, 'r': 210, 'c': 1155.0},
 11: {'p': 21, 'r': 638, 'c': 240.57352941176475},
 12: {'p': 24, 'r': 421, 'c': 563.9411764705883},
 13: {'p': 39, 'r': 245, 'c': 857.7368421052631},
 14: {'p': 4, 'r': 495, 'c': 5.012658227848078},
 15: {'p': 23, 'r': 3, 'c': 1339.0},
 16: {'p': 89, 'r': 496, 'c': 847.578947368421},
 17: {'p': 49, 'r': 374, 'c': 465.1052631578947},
 18: {'p': 5, 'r': 351, 'c': 933.0},
 19: {'p': 12, 'r': 550, 'c': 827.0},
 20: {'p': 11, 'r': 342, 'c': 16.014705882352928},
 21: {'p': 61, 'r': 667, 'c': 406.8088235294117},
 22: {'p': 31, 'r': 50, 'c': 364.0},
 23: {'p': 40, 'r':

In [68]:
taskQueue

{'m0': [28,
  22,
  42,
  43,
  4,
  6,
  5,
  35,
  24,
  23,
  47,
  37,
  50,
  44,
  18,
  27,
  15,
  10,
  19],
 'm1': [20, 8, 7, 30, 11, 29, 32, 12, 21, 26],
 'm2': [34],
 'm3': [45, 25, 36, 2, 41, 17, 33, 9, 13, 31, 16, 48, 40, 1, 3, 46, 49],
 'm4': [14, 39, 38]}

In [69]:
machines

{'m0': {'speed': '0.9583333333333334', 'readyTime': 0},
 'm1': {'speed': '0.5729166666666666', 'readyTime': 0},
 'm2': {'speed': '0.010416666666666666', 'readyTime': 0},
 'm3': {'speed': '1.0', 'readyTime': 0},
 'm4': {'speed': '0.20833333333333334', 'readyTime': 0}}

In [546]:
def calculateCost(data, taskQueue, machines):
    calculatedWeight = 0
    for key, tasks in taskQueue.items():
        print(f"m: {key}")
        timeCounter = 0
        for task in tasks:
            print(f"task: {task}, {data[task]}")
            while machines[key]['readyTime'] > timeCounter or data[task]['r'] > timeCounter:
                timeCounter += 1 
            print(f"timeCounter: {timeCounter}")
            print(f"machines[key]['readyTime']: {machines[key]['readyTime']}")
            
            machines[key]['readyTime'] = timeCounter + (data[task]['p'] / float(machines[key]['speed']) )
            
            print(f"machines[key]['readyTime']: {machines[key]['readyTime']}")
            
            data[task]['c'] = timeCounter + (data[task]['p'] / float(machines[key]['speed']) ) - data[task]['r']
            
            
            print(f"data[task]['c']: {data[task]['c']}")
            print(f"realy p: {(data[task]['p'] / float(machines[key]['speed']) )}")
            print()
        #    if timeCounter < data[task]['r']:
        #        timeCounter = data[task]['r']
            # add p time to current time
        #    timeCounter += data[task]['p']
      #      c = timeCounter
      #      data[task]['c'] = c
    
    for key, value in data.items():
        print(f"{key} -> {value}")
    # check correctness
    if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
        print("Error")

    # verify cost
    if data[task]["c"] > data[task]["d"]:
        calculatedWeight += data[task]["w"]
    #print(calculatedWeight)
    return calculatedWeight

In [547]:
motherFolder = "out" # "PTSZ-instancje"
data = {}
indicies = ['136723'] #["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    print(index)
    for samples in nSamples:
        taskQueue = {}
        with open(f'{motherFolder}/{index}/out{index}_{samples}.txt', "r") as result:
            res = result.read().split('\n')
            weight = int(res[0])
            for i, name in enumerate(machineNames):
                gen = (t for t in res[i+1].split(" ") if t.isdigit())
                taskQueue[name] = [int(t) for t in gen]
    
        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])
            machines = {}
            for name, speed in zip(machineNames, d[1].split(' ')):
                machines[name] = {'speed':speed, 'readyTime':0}
            for i,line in enumerate(d[2:]):
                if line == '':
                    continue
                p,r = line.split(' ')
                p,r = int(p), int(r)
                data[i+1] = {"p": p, "r": r, "c": 0 }
        
        calculatedWeight = calculateCost(data, taskQueue, machines)
        print(calculatedWeight)
        #if calculatedWeight != weight:
        #    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
        #else:
        #    print(f"Weight: {calculatedWeight} confirmed")
    break
    print()
    print()
        
        

136723
m: m0
task: 21, {'p': 31, 'r': 83, 'c': 0}
timeCounter: 83
machines[key]['readyTime']: 0
machines[key]['readyTime']: 143.1764705882353
data[task]['c']: 60.176470588235304
realy p: 60.1764705882353

task: 2, {'p': 67, 'r': 137, 'c': 0}
timeCounter: 144
machines[key]['readyTime']: 143.1764705882353
machines[key]['readyTime']: 274.05882352941177
data[task]['c']: 137.05882352941177
realy p: 130.05882352941177

task: 29, {'p': 65, 'r': 274, 'c': 0}
timeCounter: 275
machines[key]['readyTime']: 274.05882352941177
machines[key]['readyTime']: 401.1764705882353
data[task]['c']: 127.1764705882353
realy p: 126.1764705882353

task: 38, {'p': 37, 'r': 402, 'c': 0}
timeCounter: 402
machines[key]['readyTime']: 401.1764705882353
machines[key]['readyTime']: 473.8235294117647
data[task]['c']: 71.8235294117647
realy p: 71.82352941176471

task: 24, {'p': 33, 'r': 480, 'c': 0}
timeCounter: 480
machines[key]['readyTime']: 473.8235294117647
machines[key]['readyTime']: 544.0588235294117
data[task]['c']:

KeyError: 'd'

In [66]:
machines

{'m0': {'speed': '0.9583333333333334', 'readyTime': 0},
 'm1': {'speed': '0.5729166666666666', 'readyTime': 0},
 'm2': {'speed': '0.010416666666666666', 'readyTime': 0},
 'm3': {'speed': '1.0', 'readyTime': 0},
 'm4': {'speed': '0.20833333333333334', 'readyTime': 0}}

In [11]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# TaskScheduler

In [106]:
a,b,c,d = (1,1,1,1)
totalCost = 0
data = {}

indicies = ['136723']#["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    for samples in nSamples:
        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])
            machines = {}
            for name, speed in zip(machineNames, d[1].split(' ')):
                machines[name] = {'speed':speed, 'readyTime':0, 'gain':0}
            for i,line in enumerate(d[2:]):
                if line == '':
                    continue
                p,r = line.split(' ')
                p,r = int(p), int(r)
                data[i+1] = {"p": p, "r": r, "c": 0, "penalty": 0 }
            dataCopy = copy.deepcopy(data)
        
        calculatedWeight = 0
        timeCounter = 0
        outSchedule = {}
        for name in machineNames:
            outSchedule[name] = []
        
        start = time.time()
        while len(data) > 0:
            readyToGo = {}
            for key, task in data.items():
                if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                    endTime = timeCounter+task['p']
                    penaltyEndTask = ( endTime if endTime > 0 else 0 ) * 1
                    penaltyPersistance = task['p']
                    task['penalty'] = a*penaltyEndTask + c*penaltyPersistance
                    readyToGo[key] = task
                
            if len(readyToGo) == 0: # currently there is no task to go
                #print("upsi readyToGo")
                timeCounter += 1
                continue
            #print(f"readyToGo: {readyToGo}")

            # select task
            readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty']), reverse=True)
            selectedTask = readyToGoSorted[0]
            
            # select machine
            readyMachines = [d for d in zip(machines.keys(), machines.values() ) if d[1]['readyTime'] <= timeCounter]
            if len(readyMachines) == 0: # currently there is free machine
                #print("upsi readyMachines")
                timeCounter += 1
                continue
            
            
            for name, machine in machines.values():
                howLongItWillBeDisabled = 0 ##########################################
                speed = machines[name]['speed']
                
                machines[name]['gain'] = speed
            selectedMachine = max(readyMachines, key=lambda y: (y[1]['speed']))
            
            print(f"timeCounter: {timeCounter}")
            print(f"selectedTask: {selectedTask}")
            
            # remove task from list active tasks, add time, append outschedule
            data.pop(selectedTask[0])
            
            print(f"selectedMachine: {selectedMachine}")
            #timeCounter += 1
            machines[selectedMachine[0]]['readyTime'] = timeCounter + (selectedTask[1]['p'] / float(selectedMachine[1]['speed']) )
            print(machines)
            
            print()
            
            outSchedule[selectedMachine[0]].append(selectedTask[0])    
        
        end = time.time()
        #print(end-start)
        
        #cost = calculateCost(dataCopy, outSchedule)
        cost = 5
        #print(cost)
        totalCost += cost
        
        print(outSchedule)
        
        with open(f'out/{index}/out{index}_{samples}.txt', "w") as f:
            f.write(f"{cost}\n")         
                #f.write(f"{' '.join([str(x) for x in line])}\n")
            for name in machineNames:
                f.write(f"{' '.join([str(x) for x in outSchedule[name]])}\n")
        
        break
        
print(totalCost)

timeCounter: 0
selectedTask: (28, {'p': 89, 'r': 0, 'c': 0, 'penalty': 178})
selectedMachine: ('m4', {'speed': '1.0', 'readyTime': 0})
{'m0': {'speed': '0.5151515151515151', 'readyTime': 0}, 'm1': {'speed': '0.5252525252525253', 'readyTime': 0}, 'm2': {'speed': '0.7777777777777778', 'readyTime': 0}, 'm3': {'speed': '0.797979797979798', 'readyTime': 0}, 'm4': {'speed': '1.0', 'readyTime': 89.0}}

timeCounter: 29
selectedTask: (13, {'p': 71, 'r': 29, 'c': 0, 'penalty': 171})
selectedMachine: ('m3', {'speed': '0.797979797979798', 'readyTime': 0})
{'m0': {'speed': '0.5151515151515151', 'readyTime': 0}, 'm1': {'speed': '0.5252525252525253', 'readyTime': 0}, 'm2': {'speed': '0.7777777777777778', 'readyTime': 0}, 'm3': {'speed': '0.797979797979798', 'readyTime': 117.9746835443038}, 'm4': {'speed': '1.0', 'readyTime': 89.0}}

timeCounter: 49
selectedTask: (31, {'p': 50, 'r': 49, 'c': 0, 'penalty': 149})
selectedMachine: ('m2', {'speed': '0.7777777777777778', 'readyTime': 0})
{'m0': {'speed': '

In [33]:
x = {'m0':{'speed':2, 'readyTime':0},'m1':{'speed':3, 'readyTime':0},'m2':{'speed':1, 'readyTime':5}, }

In [18]:
zip(machines.keys(), machines.values() )

In [37]:
[d for d in x.values() if d['readyTime'] <= 0]

[{'speed': 2, 'readyTime': 0}, {'speed': 3, 'readyTime': 0}]

In [42]:
readyMachines = [d for d in x.values() if d['readyTime'] <= 0]
readyMachines = max(readyMachines, key=lambda y: (y['speed']))
readyMachines

{'speed': 3, 'readyTime': 0}

In [9]:
from scipy.optimize import minimize

In [10]:
x0 = (1, 1, 1, 1)

In [17]:
def costFunction(x):
    a,b,c,d = x
    totalCost = 0
    data = {}
    indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
    for index  in indicies:
        for samples in nSamples:
            with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
                d = instance.read().split('\n')
                n = int(d[0])                
                for i,line in enumerate(d[1:]):
                    if line == '':
                        continue
                    p,r,d,w = line.split(' ')
                    p,r,d,w = int(p), int(r), int(d), int(w)
                    data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0, "penalty": 0 }
                dataCopy = copy.deepcopy(data)


            calculatedWeight = 0
            timeCounter = 0
            outSchedule = []

            start = time.time()
            while len(data) > 0:
                readyToGo = {}
                for key, task in data.items():
                    if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                        endTime = timeCounter+task['p']-task['d']
                        penaltyEndTask = ( endTime if endTime > 0 else 0 ) * task['w']
                        penaltyWeight = task['w']
                        penaltyPersistance = task['p']
                        penaltyDue = task['d'] - timeCounter
                        task['penalty'] = a*penaltyEndTask + b*penaltyWeight + c*penaltyPersistance - d*penaltyDue
                        readyToGo[key] = task

                if len(readyToGo) == 0: # currently there is no task to go
                    #print("upsi")
                    timeCounter += 1
                    continue
                #print(f"readyToGo: {readyToGo}")

                readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['w']), reverse=True)
                selectedTask = readyToGoSorted.pop(0)
                #print(timeCounter)
                #print(f"selectedTask: {selectedTask}")

                data.pop(selectedTask[0])
                timeCounter += selectedTask[1]['p']
                outSchedule.append(selectedTask[0])    

            end = time.time()
          #  print(int((end-start)*1000) if int((end-start)*1000) != 0 else 1)

            cost = calculateCost(dataCopy, outSchedule)
            #print(cost)
            totalCost += cost
    print(f"[{totalCost},{a},{b},{c},{d}],")
    return totalCost

In [18]:
options = {"maxiter":40, "disp":True}

In [23]:
res = minimize(costFunction, x0, method='TNC', options=options, tol=1e-2)

[499007,1.0,1.0,1.0,1183],
[499007,1.0,1.0,1.0,1183],
[499012,1.00000001,1.0,1.0,1183],
[499007,1.0,1.00000001,1.0,1183],
[499030,1.0,1.0,1.00000001,1183],
[499007,1.0,1.0,1.0,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,1.0000000036691032,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.00000001,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999980877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,1.0000000036691032,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.00000001,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999980877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999968345517,1.0,0.9999999854389375,1183],
[499007,0.9999999968345517,1.0,0.9999999854389375,1183],
[499007,1.0000000

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = [1,2,3]

In [ ]:
plt.plot(x)
plt.title("Minimalizacja funkcji celu")
plt.xaxis("iteracja")
plt.yaxis("Koszt")

In [94]:
costFunction(x)

1
7
13
25
17
19
26
33
38
61
1
4
10
17
29
42
64
85
99
140
1
4
11
19
30
45
63
152
111
133
1
3
15
16
25
32
48
61
82
88
1
1
6
13
30
36
48
64
94
108
1
3
6
11
19
27
38
52
62
79
1
3
8
22
29
38
56
71
105
124
1
4
10
17
26
39
53
69
105
107
1
1
1
2
2
4
7
6
9
10
1
3
6
10
16
26
33
54
57
68
1
3
7
15
22
45
45
56
75
108
1
1
1
1
2
4
5
7
8
11
1
4
11
21
32
47
74
87
110
146
1
3
5
8
14
28
34
39
54
84


385570

In [85]:
x = (-140036.8041378972, 2.5486165590751364, -0.1998934445512417, 1183)